In [1]:
# Install PySpark (if not already installed)
# !pip install pyspark

# Import SparkSession from pyspark
from pyspark.sql import SparkSession

# Create a local Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Module5Homework") \
    .getOrCreate()

# Print the Spark version
print("Spark Version:", spark.version)


25/03/10 20:27:17 WARN Utils: Your hostname, multisrf2 resolves to a loopback address: 127.0.1.1; using 192.168.1.138 instead (on interface wlp47s0)
25/03/10 20:27:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 20:27:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/10 20:27:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Version: 3.5.5


In [2]:
# Download the Parquet data (this command runs in the shell)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet



--2025-03-10 20:27:59--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
3.160.226.161, 3.160.226.85, 3.160.226.111, ...vzurychx.cloudfront.net)... 
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.226.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  16.6MB/s    in 3.9s    

2025-03-10 20:28:03 (15.7 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [3]:
# Read the Parquet file into a Spark DataFrame
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
# Repartition the DataFrame into 4 partitions
df_repart = df.repartition(4)


In [4]:
# Write the repartitioned DataFrame back to Parquet format
df_repart.write.mode("overwrite").parquet("yellow_tripdata_2024-10_repart.parquet")

In [5]:
!du -sh yellow_tripdata_2024-10_repart.parquet


91M	yellow_tripdata_2024-10_repart.parquet


In [ ]:
## QUESTION 2

In [8]:
!ls -lh yellow_tripdata_2024-10_repart.parquet/*.parquet | awk '{sum += $5} END {print sum/NR}'

23


In [12]:
import os
import glob

# Directory where the parquet files were written
output_dir = "yellow_tripdata_2024-10_repart.parquet"

# Find all .parquet files (ignore .crc files)
parquet_files = glob.glob(os.path.join(output_dir, "*.parquet"))

# Calculate the total size of the .parquet files
total_size_bytes = 0
for f in parquet_files:
    total_size_bytes += os.path.getsize(f)

# Compute average size in MB
if len(parquet_files) > 0:
    avg_size_mb = total_size_bytes / len(parquet_files) / (1024 * 1024)
    print(f"Average Parquet file size: {avg_size_mb:.2f} MB")
else:
    print("No .parquet files found.")


Average Parquet file size: 22.39 MB


In [13]:
## question 3


In [14]:
from pyspark.sql.functions import to_date, col

# Let's assume 'df' is the DataFrame containing your taxi data
columns_lower = [c.lower() for c in df.columns]
pickup_candidates = [df.columns[i] for i, c in enumerate(columns_lower) if "pickup" in c]

if len(pickup_candidates) == 0:
    raise ValueError("No column found with the word 'pickup' in its name.")
elif len(pickup_candidates) > 1:
    raise ValueError(f"Multiple columns found containing 'pickup': {pickup_candidates}")
else:
    pickup_col = pickup_candidates[0]
    print(f"Using pickup column: {pickup_col}")

# Now filter for October 15th (2024-10-15)
df_oct15 = df.filter(to_date(col(pickup_col)) == "2024-10-15")

# Count records
trip_count_oct15 = df_oct15.count()
print("Number of trips on October 15th:", trip_count_oct15)


Using pickup column: tpep_pickup_datetime
Number of trips on October 15th: 128893


In [15]:
columns_lower


['vendorid',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'ratecodeid',
 'store_and_fwd_flag',
 'pulocationid',
 'dolocationid',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [ ]:
#limit to those that finish same day

In [21]:
from pyspark.sql.functions import to_date, col

# Assume your DataFrame is called df

# 1) Dynamically detect the pickup column name by searching for "pickup" in df.columns
columns_lower = [c.lower() for c in df.columns]
pickup_candidates = [df.columns[i] for i, c in enumerate(columns_lower) if "pickup" in c]

if len(pickup_candidates) == 0:
    raise ValueError("No column found with the word 'pickup' in its name.")
elif len(pickup_candidates) > 1:
    raise ValueError(f"Multiple columns found containing 'pickup': {pickup_candidates}")
else:
    pickup_col = pickup_candidates[0]
    print(f"Using pickup column: {pickup_col}")

# 2) Filter for trips that started AND ended on 2024-10-15
df_oct15 = df.filter(
    (to_date(col(pickup_col)) == "2024-10-15") &
    (to_date(col("tpep_dropoff_datetime")) == "2024-10-15")
)

# # 3) Show the filtered records (or count them)
# df_oct15.show()  # or df_oct15.count()


Using pickup column: tpep_pickup_datetime


In [22]:
# Count records
trip_count_oct15 = df_oct15.count()
print("Number of trips on October 15th:", trip_count_oct15)


Number of trips on October 15th: 127993


In [ ]:
## question 4 Longest trip in hours

In [16]:
from pyspark.sql.functions import unix_timestamp, max as spark_max

# Create a new column for trip duration in hours
df_duration = df.withColumn("duration_hours", 
                            (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 3600)

# Find the longest trip (maximum duration)
max_duration = df_duration.select(spark_max("duration_hours")).collect()[0][0]
print("Longest trip duration (hours):", max_duration)


Longest trip duration (hours): 162.61777777777777


In [17]:
## question 6

In [23]:
# Download the taxi zone lookup CSV file
# !wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

# Load the CSV into a Spark DataFrame
zone_df = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)
zone_df.createOrReplaceTempView("zones")

# Create or use a temporary view for the Yellow taxi data if not already done
df.createOrReplaceTempView("yellow")

# Use a Spark SQL query to join the data, count trips per zone, and select the least frequent one.
least_frequent_zone = spark.sql("""
SELECT z.zone, COUNT(*) as trip_count
FROM yellow y
JOIN zones z ON y.PULocationID = z.LocationID
GROUP BY z.zone
ORDER BY trip_count ASC
""")
#LIMIT 1

least_frequent_zone.show()


--2025-03-10 21:01:04--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
3.160.226.111, 3.160.226.228, 3.160.226.85, ...vzurychx.cloudfront.net)... 
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.226.111|:443... connected.
200 OKequest sent, awaiting response... 
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-10 21:01:05 (15.5 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]

+--------------------+----------+
|                zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
|       Rikers Island|         2|
|       Arden Heights|         2|
|         Jamaica Bay|         3|
| Green-Wood Cemetery|         3|
|Charleston/Totten...|         4|
|   Rossville/Woodrow|         4|
|       West Brighton|         4|
|Eltingville/Annad...|         4|
|       Port Richmond|         4|
|         Grea